## Roberta fine-tuning + hyperparameter tuning

#### Setup

In [1]:
!pip install -q optuna
!pip install -q evaluate
!pip install -q emoji==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import (
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainerCallback,
    EarlyStoppingCallback
)
import evaluate
import torch.nn.functional as F
import optuna
import wandb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
wandb.login(key="<wandb key>")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mayachn3 (mayachn3-maya-bondar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Data Preparation
### Load Dataset

In [6]:
train = pd.read_csv("OOT_train.csv", encoding='latin-1')
val = pd.read_csv("OOT_val.csv", encoding='latin-1')
test = pd.read_csv("OOT_test.csv", encoding='latin-1')


In [7]:
# train = train.head(1000)
# val = val.head(1000)
# test = test.head(1000)

### Preprocessing

In [8]:
#encoding the labels numerically from Sentiment

ordinal_mapping = {
    'Extremely Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Extremely Positive': 4
}

# map to ordinal labels
train["ordinal_label_id"] = train["Sentiment"].map(ordinal_mapping)
val["ordinal_label_id"] = val["Sentiment"].map(ordinal_mapping)
test["ordinal_label_id"] = test["Sentiment"].map(ordinal_mapping)



In [9]:
# Concat the relevant columns into one string with seperation.
# for example: "Tweet: my food stock is low | Location: Canada | Date: 2020-03-17 | URL: https://t.co/abcd"

# Function to build the input string from multiple columns
def build_augmented_input(row):
    parts = []

    if pd.notna(row.get('clean_tweet')):
        parts.append(f"{row['clean_tweet']}")

    if pd.notna(row.get('Location_standardized')) and row['Location_standardized'].lower() != 'unknown':
        parts.append(f"{row['Location_standardized']}")

    if pd.notna(row.get('TweetAt')):
        parts.append(f"{row['TweetAt']}")


    return " | ".join(parts)

# Apply to the DataFrames
train['model_input'] = train.apply(build_augmented_input, axis=1)
val['model_input'] = val.apply(build_augmented_input, axis=1)
test['model_input'] = test.apply(build_augmented_input, axis=1)

# Create  new DataFrames with only what's needed for modeling
formatted_train = train[['model_input', 'ordinal_label_id']].copy()
formatted_val = val[['model_input', 'ordinal_label_id']].copy()
formatted_test = test[['model_input', 'ordinal_label_id']].copy()



In [10]:
def balance_dataset(df, target_samples_per_class=5000):
    """Balance dataset by undersampling"""
    balanced_dfs = []

    print("Original class distribution:")
    print(df['ordinal_label_id'].value_counts().sort_index())

    for class_id in range(5):
        class_data = df[df['ordinal_label_id'] == class_id]

        if len(class_data) > target_samples_per_class:
            class_data = class_data.sample(n=target_samples_per_class, random_state=42)
            print(f"Class {class_id}: {len(class_data)} samples (undersampled)")
        else:
            print(f"Class {class_id}: {len(class_data)} samples (kept all)")

        balanced_dfs.append(class_data)

    balanced_df = pd.concat(balanced_dfs, ignore_index=True).sample(frac=1, random_state=42)

    print(f"Balanced dataset: {len(balanced_df)} total samples")
    print("New distribution:")
    print(balanced_df['ordinal_label_id'].value_counts().sort_index())

    return balanced_df

# Apply balancing to training data
formatted_train = balance_dataset(formatted_train, target_samples_per_class=5000)

Original class distribution:
ordinal_label_id
0     5175
1     9230
2     6784
3    10140
4     5845
Name: count, dtype: int64
Class 0: 5000 samples (undersampled)
Class 1: 5000 samples (undersampled)
Class 2: 5000 samples (undersampled)
Class 3: 5000 samples (undersampled)
Class 4: 5000 samples (undersampled)
Balanced dataset: 25000 total samples
New distribution:
ordinal_label_id
0    5000
1    5000
2    5000
3    5000
4    5000
Name: count, dtype: int64


### Tokenization

## Model
### Define Model

In [11]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_data(data, max_length=128):
    return tokenizer(
        data['model_input'].tolist(),
        truncation=True,
        padding=False,
        max_length=max_length,
        add_special_tokens=True,
        return_attention_mask=True,
        return_token_type_ids=False
    )

train_encodings = tokenize_data(formatted_train)
val_encodings = tokenize_data(formatted_val)
test_encodings = tokenize_data(formatted_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [12]:
## define a PyTorch Dataset
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # Should be integers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # For training
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to integers if not already
train_labels = formatted_train['ordinal_label_id'].tolist()
val_labels = formatted_val['ordinal_label_id'].tolist()
test_labels = formatted_test['ordinal_label_id'].tolist()


train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)


In [13]:

# define mapping between label id and sentiment for later use and conveniency
ordinal_label2id = ordinal_mapping
ordinal_id2label = {v: k for k, v in ordinal_mapping.items()}


In [14]:
def compute_detailed_metrics(eval_pred):
    """Enhanced metrics using HuggingFace Evaluate library"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Load HuggingFace metrics (cached after first load)
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")

    # Compute standard classification metrics
    results = {}

    # Basic metrics
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='weighted'))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average='macro'))

    # Per-class F1 scores (HF doesn't have this built-in, so keep custom)
    f1_per_class = f1_score(labels, predictions, average=None)
    for i, class_name in enumerate(['extremely_negative', 'negative', 'neutral', 'positive', 'extremely_positive']):
        results[f'f1_{class_name}'] = f1_per_class[i]

        # Per-class precision and recall
        precision_per_class = precision_score(labels, predictions, average=None, zero_division=0)
        recall_per_class = recall_score(labels, predictions, average=None, zero_division=0)
        results[f'precision_{class_name}'] = precision_per_class[i]
        results[f'recall_{class_name}'] = recall_per_class[i]

        # Per-class accuracy
        class_mask = (labels == i)
        if class_mask.sum() > 0:
            results[f'accuracy_{class_name}'] = accuracy_score(labels[class_mask], predictions[class_mask])
        else:
            results[f'accuracy_{class_name}'] = 0.0

    # Custom ordinal metrics (HF doesn't have these)
    results['mae'] = np.mean(np.abs(predictions - labels))
    results['adjacent_accuracy'] = np.sum(np.abs(predictions - labels) <= 1) / len(labels)

    # Quadratic Weighted Kappa (custom)
    from sklearn.metrics import cohen_kappa_score
    try:
        qwk = cohen_kappa_score(labels, predictions, weights='quadratic')
        results['quadratic_weighted_kappa'] = qwk
    except:
        results['quadratic_weighted_kappa'] = 0.0

    return results

In [15]:
def find_optimal_batch_size(base_batch_size):
    """Find the largest batch size that fits in GPU memory"""
    if device.type == "cpu":
        return base_batch_size

    # Try larger batch sizes for GPU
    for multiplier in [4, 3, 2, 1]:
        try_batch_size = base_batch_size * multiplier
        try:
            # Test if this batch size fits
            dummy_input = torch.randn(try_batch_size, 128, 768, device=device)
            dummy_output = torch.randn(try_batch_size, 5, device=device)
            del dummy_input, dummy_output
            torch.cuda.empty_cache() if device.type == "cuda" else None
            return try_batch_size
        except RuntimeError:  # Out of memory
            continue
    return base_batch_size

In [16]:
class SimpleMetricsLogger(TrainerCallback):
    """Simple callback to log detailed metrics every epoch"""

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or not wandb.run:
            return

        # Only log when we have evaluation metrics (after each epoch)
        if 'eval_loss' in logs:
            current_epoch = int(state.epoch)

            # Get current learning rate
            current_lr = args.learning_rate
            if 'learning_rate' in logs:
                current_lr = logs['learning_rate']

            #Get training loss from state history
            train_loss = 0
            if state.log_history:
                # Find the most recent training loss
                for log_entry in reversed(state.log_history):
                    if 'train_loss' in log_entry:
                        train_loss = log_entry['train_loss']
                        break

            detailed_metrics = {
                "Epoch": current_epoch,
                "Stage": 1,
                "Unfrozen_Layers": 12,
                "Train Loss": train_loss,
                "Train Accuracy": 0,  # Usually not computed during training
                "Validation Loss": logs.get('eval_loss', 0),
                "Validation Accuracy": logs.get('eval_accuracy', 0),
                "Validation Precision": logs.get('eval_precision_macro', 0),
                "Validation Recall": logs.get('eval_recall_macro', 0),
                "Validation F1": logs.get('eval_f1_macro', 0),
                "Validation MAE": logs.get('eval_mae', 0),
                "Validation Adjacent Accuracy": logs.get('eval_adjacent_accuracy', 0),
                "Validation QWK": logs.get('eval_quadratic_weighted_kappa', 0),
                "Learning_Rate": current_lr,
            }

            # Log to WandB
            wandb.log(detailed_metrics)

            #Print progress to console
            print(f"Epoch {current_epoch}: "
                  f"Train Loss: {train_loss:.4f}, "  # ← Now shows real values
                  f"Val Loss: {logs.get('eval_loss', 0):.4f}, "
                  f"Val F1: {logs.get('eval_f1_macro', 0):.4f}, "
                  f"QWK: {logs.get('eval_quadratic_weighted_kappa', 0):.4f}")

In [17]:
def save_training_checkpoint(model, optimizer, epoch, loss, trial_params, filepath, trial_number, current_score, trainer):
    """Save complete training checkpoint and handle best model updates"""
    global best_score, best_model_path

    # Get the trial directory from filepath
    trial_dir = os.path.dirname(filepath)

    # Save trial checkpoint
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        # 'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'trial_params': trial_params,
        'model_config': model.config.to_dict(),
        'tokenizer_name': model_name,
        'current_score': current_score,  # Add score to checkpoint
        'trial_number': trial_number,
    }

    # Save all trial files in the same directory
    torch.save(checkpoint, filepath)
    torch.save(model.state_dict(), os.path.join(trial_dir, 'model_roberta_weights.pt'))
    torch.save(model, os.path.join(trial_dir, 'model_roberta.pt'))

    print(f"✅ Trial checkpoint saved: {filepath}")
    print(f"✅ Model files saved in: {trial_dir}")

    # Update best model if needed
    if current_score > best_score:
        best_score = current_score

        # Save HuggingFace format to best model directory
        trainer.save_model(best_model_path)

        # Also save our custom format in best model directory
        os.makedirs(best_model_path, exist_ok=True)
        best_checkpoint_path = os.path.join(best_model_path, 'best_checkpoint.ckpt')
        best_weights_path = os.path.join(best_model_path, 'model_roberta_weights.pt')
        best_model_file_path = os.path.join(best_model_path, 'model_roberta.pt')

        torch.save(checkpoint, best_checkpoint_path)
        torch.save(model.state_dict(), best_weights_path)
        torch.save(model, best_model_file_path)

        print(f"🏆 New best model saved! Score: {current_score:.4f} (Trial {trial_number})")
        print(f"🏆 Best model files saved in: {best_model_path}")

        # Optional: Log to W&B
        # wandb.log({
        #     "best_score_so_far": current_score,
        #     "best_trial_number": trial_number,
        # })
    else:
        print(f"📊 Trial {trial_number} score: {current_score:.4f} (Best: {best_score:.4f})")

In [18]:
# Global variables to track best model
best_score = 0.0
best_model_path = "./best_roberta_model_so_far"

### Hyperparameter Tuning

In [19]:
def objective(trial):
    """Clean, organized objective function for Optuna hyperparameter optimization"""
    global best_score, best_model_path

    # === GPU MEMORY CLEANUP ===
    if device.type == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # === HYPERPARAMETER SAMPLING ===
    # Core training parameters
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 5e-4, log=True)
    base_batch_size = trial.suggest_categorical("batch_size", [16,32,64])
    label_smoothing = trial.suggest_float("label_smoothing", 0.05, 0.15)
    num_epochs = trial.suggest_int("num_epochs", 10, 15)

    # Advanced optimization parameters
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.05, 0.15)
    weight_decay = trial.suggest_float("weight_decay", 0.05, 0.15)

    # Model architecture parameters
    attention_dropout = trial.suggest_float("attention_dropout", 0.3, 0.4)
    hidden_dropout = trial.suggest_float("hidden_dropout", 0.3, 0.4)

    #R-drop parameter
    # rdrop_alpha = trial.suggest_float("rdrop_alpha", 0.0, 1.0)

    # PRINT CHOSEN PARAMETERS
    print(f"TRIAL {trial.number} - TESTING THESE PARAMETERS:")
    print(f"Learning Rate:      {learning_rate:.2e}")
    print(f"Epochs:             {num_epochs}")
    print(f"Warmup Ratio:       {warmup_ratio:.3f}")
    print(f"Weight Decay:       {weight_decay:.3f}")
    print(f"Attention Dropout:  {attention_dropout:.3f}")
    print(f"Hidden Dropout:     {hidden_dropout:.3f}")


    # Optimize batch size for available hardware
    batch_size = find_optimal_batch_size(base_batch_size)


    # === EXPERIMENT TRACKING SETUP ===
    wandb.init(
        project="covid-tweet-sentiment-hf-roberta-regularloss",
        name=f"trial_{trial.number}",
        config={
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "warmup_ratio": warmup_ratio,
            "weight_decay": weight_decay,
            "attention_dropout": attention_dropout,
            "hidden_dropout": hidden_dropout
            # "rdrop_alpha": rdrop_alpha ,
        },
        reinit=True
    )

    try:
        # === MODEL SETUP ===
        model = _setup_model(attention_dropout, hidden_dropout)

        # === TRAINING CONFIGURATION ===
        training_args = _create_training_args(
            trial_number=trial.number,
            learning_rate=learning_rate,
            batch_size=batch_size,
            num_epochs=num_epochs,
            warmup_ratio=warmup_ratio,
            weight_decay=weight_decay,
            label_smoothing_factor=label_smoothing
        )

        trial_params = {
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "warmup_ratio": warmup_ratio,
            "weight_decay": weight_decay,
            "attention_dropout": attention_dropout,
            "hidden_dropout": hidden_dropout
        }

        trainer = _create_trainer(model, training_args, trial.number, trial_params)
        trainer.train()

        # Set Checkpoint per trial
        checkpoint_dir = f"./checkpoints_roberta/trial_{trial.number}"
        os.makedirs(checkpoint_dir, exist_ok=True)

        final_epoch = int(trainer.state.epoch)
        checkpoint_path = f"{checkpoint_dir}/final_epoch_{final_epoch}.ckpt"

        # Get final training loss
        final_loss = 0
        if trainer.state.log_history:
            for log_entry in reversed(trainer.state.log_history):
                if 'train_loss' in log_entry:
                    final_loss = log_entry['train_loss']
                    break

        eval_results = trainer.evaluate()
        current_score = eval_results["eval_quadratic_weighted_kappa"]

        save_training_checkpoint(
            model=trainer.model,
            optimizer=None,
            epoch=final_epoch,
            loss=final_loss,
            trial_params=trial_params,
            filepath=checkpoint_path,
            trial_number=trial.number,
            current_score=current_score,
            trainer=trainer
        )

        # Log GPU usage if available
        if device.type == "cuda":
            print(f"GPU Memory Used: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")

        return current_score

    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        raise optuna.exceptions.TrialPruned()

    finally:
        # === CLEANUP ===
        if device.type == "cuda":
            torch.cuda.empty_cache()
        wandb.finish()


def _setup_model(attention_dropout, hidden_dropout):
    """Setup and configure the model with dropout and freezing"""
    model = AutoModelForSequenceClassification.from_pretrained(
        "cardiffnlp/twitter-roberta-base-sentiment",
        num_labels=5,
        id2label=ordinal_id2label,
        label2id=ordinal_label2id,
        ignore_mismatched_sizes=True
    )

    # Apply dropout configuration
    model.config.attention_probs_dropout_prob = attention_dropout
    model.config.hidden_dropout_prob = hidden_dropout

    # GPU optimizations
    if device.type == "cuda":
        model.gradient_checkpointing_enable()

        model.to(device)

    return model


def _create_training_args(trial_number, learning_rate, batch_size, num_epochs, warmup_ratio, weight_decay, label_smoothing_factor=0.1):
    """Create optimized training arguments"""
    return TrainingArguments(
        # output_dir=f"./results/trial_{trial_number}",

        # Core training parameters
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,  # Larger eval batch
        learning_rate=learning_rate,
        label_smoothing_factor=label_smoothing_factor,

        # Learning rate scheduling
        lr_scheduler_type="cosine",
        warmup_ratio=warmup_ratio,

        # Optimization
        optim="adamw_torch",
        weight_decay=weight_decay,
        max_grad_norm=1.0,

        # Evaluation and saving
        eval_strategy="epoch",
        save_strategy="no",
        save_total_limit=1,
        # load_best_model_at_end=True,
        metric_for_best_model="eval_quadratic_weighted_kappa",
        greater_is_better=True,

        # Performance optimizations
        fp16=device.type == "cuda",
        # tf32=device.type == "cuda",
        dataloader_pin_memory=True,
        dataloader_persistent_workers=device.type == "cuda",
        dataloader_num_workers=2 if device.type == "cuda" else 0,
        dataloader_drop_last=False,
        group_by_length=True,
        gradient_accumulation_steps=1,
        dataloader_prefetch_factor=2 if device.type == "cuda" else None,

        # Logging
        logging_steps=100,
        report_to="wandb",
        remove_unused_columns=False,

        # Evaluation optimizations
        eval_accumulation_steps=None,
        prediction_loss_only=False,

    )

def _create_trainer(model, training_args, trial_number, trial_params):
    """ordinal loss trainer"""

    callbacks = [
        EarlyStoppingCallback(early_stopping_patience=2),
        SimpleMetricsLogger(),
        ]

    return Trainer(  # Uses ordinal loss always
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_detailed_metrics,
        tokenizer=tokenizer,
        callbacks=callbacks,
    )

In [20]:
from datetime import datetime
import json

def save_best_hyperparameters(study, model_name="roberta"):
    """Save the best hyperparameters found by Optuna"""

    # Check if any trial completed successfully
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    if not completed_trials:
        print("No completed trials found. Skipping saving best hyperparameters.")
        return None

    best_params = study.best_params
    best_score = study.best_value

    # Save to JSON file
    results = {
        "model_name": model_name,
        "best_score": best_score,
        "best_params": best_params,
        "timestamp": str(datetime.now()),
        "total_trials": len(study.trials),
        "completed_trials": len(completed_trials)
    }

    filename = f"best_params_{model_name}.json"
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"Best hyperparameters saved to {filename}")
    print(f"Best score: {best_score:.4f}")
    print(f"Best params: {best_params}")
    print(f"Completed {len(completed_trials)}/{len(study.trials)} trials")

    return results

## Training

In [21]:
# Run optimization

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=8)

roberta_results = save_best_hyperparameters(study, "roberta")


[I 2025-08-01 12:44:54,324] A new study created in memory with name: no-name-003c7642-7dcd-46d2-b3e5-e4293adf27cb


TRIAL 0 - TESTING THESE PARAMETERS:
Learning Rate:      2.92e-04
Epochs:             14
Warmup Ratio:       0.073
Weight Decay:       0.066
Attention Dropout:  0.386
Hidden Dropout:     0.380


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 10: Train Loss: 0.7548, Val Loss: 0.8985, Val F1: 0.0000, QWK: 0.9211
✅ Trial checkpoint saved: ./checkpoints_roberta/trial_7/final_epoch_10.ckpt
✅ Model files saved in: ./checkpoints_roberta/trial_7
📊 Trial 7 score: 0.9211 (Best: 0.9298)
GPU Memory Used: 4.02 GB


Epoch,▁▂▃▃▄▅▆▆▇██
Learning_Rate,▁▁▁▁▁▁▁▁▁▁▁
Stage,▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁
Train Loss,▁▁▁▁▁▁▁▁▁▁█
Unfrozen_Layers,▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▄▇█▇██████
Validation Adjacent Accuracy,▁▃▅▇▇▇█████
Validation F1,▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▅▂▁▂▂▁▂▂▂▂
Validation MAE,█▅▃▂▂▁▁▁▁▁▁


[I 2025-08-01 14:36:47,626] Trial 7 finished with value: 0.9210994149655075 and parameters: {'learning_rate': 5.912385786218203e-05, 'batch_size': 64, 'label_smoothing': 0.1340152351552511, 'num_epochs': 11, 'warmup_ratio': 0.13457532845956638, 'weight_decay': 0.07983415554786308, 'attention_dropout': 0.3533393048077859, 'hidden_dropout': 0.32510017157630366}. Best is trial 2 with value: 0.9297840127669785.


Best hyperparameters saved to best_params_roberta.json
Best score: 0.9298
Best params: {'learning_rate': 0.0002410038500292598, 'batch_size': 32, 'label_smoothing': 0.09079127901743848, 'num_epochs': 11, 'warmup_ratio': 0.14435680747215307, 'weight_decay': 0.0849441280799004, 'attention_dropout': 0.3758622805072069, 'hidden_dropout': 0.3533269312284606}
Completed 8/8 trials


In [22]:
import time
import os

def calculate_model_metrics(model_path="./best_model_so_far"):
    """Calculate inference time and model size"""

    # 1. Calculate Model Size
    def get_model_size_mb(path):
        total_size = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total_size += os.path.getsize(filepath)
        return total_size / (1024 * 1024)  # Convert to MB

    model_size_mb = get_model_size_mb(model_path)

    # 2. Calculate Inference Time
    def measure_inference_time():
        # Load model for timing
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer_for_timing = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

        if device.type == "cuda":
            model = model.to(device)
        model.eval()

        # Sample text for timing
        sample_text = "COVID vaccines are helping to reduce hospitalizations significantly."

        # Warm-up runs (don't count these)
        for _ in range(3):
            inputs = tokenizer_for_timing(sample_text, return_tensors="pt", truncation=True, padding=True)
            if device.type == "cuda":
                inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                _ = model(**inputs)

        # Actual timing runs
        times = []
        num_runs = 10

        for _ in range(num_runs):
            inputs = tokenizer_for_timing(sample_text, return_tensors="pt", truncation=True, padding=True)
            if device.type == "cuda":
                inputs = {k: v.to(device) for k, v in inputs.items()}

            start_time = time.time()
            with torch.no_grad():
                _ = model(**inputs)
            end_time = time.time()

            times.append(end_time - start_time)

        # Return average inference time
        avg_inference_time = sum(times) / len(times)
        return avg_inference_time

    inference_time = measure_inference_time()

    return model_size_mb, inference_time

# Load the best model
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)

# Create trainer for test evaluation
test_trainer = Trainer(
    model=best_model,
    args=TrainingArguments(
        output_dir="./temp",
        per_device_eval_batch_size=16,
        remove_unused_columns=False,
    ),
    tokenizer=tokenizer,
    compute_metrics=compute_detailed_metrics,  # Use the enhanced function!
)

# Evaluate on test set
test_results = test_trainer.evaluate(test_dataset)

# Calculate performance metrics
model_size_mb, inference_time_sec = calculate_model_metrics(best_model_path)

print("FINAL TEST SET EVALUATION - COMPREHENSIVE ANALYSIS")

# Standard Classification Metrics
print(test_results)
print("\nOVERALL CLASSIFICATION METRICS:")
print(f"Accuracy:           {test_results['eval_accuracy']:.4f}")
print(f"Validation QWK:     {test_results['eval_quadratic_weighted_kappa']:.4f}")
print(f"F1:                 {test_results['eval_f1']:.4f}")
print(f"Precision-Macro:    {test_results['eval_precision']:.4f}")
print(f"Recall-Macro:       {test_results['eval_recall']:.4f}")

# Ordinal-Aware Metrics
print("\nORDINAL-AWARE METRICS:")
print(f"Mean Absolute Error:        {test_results['eval_mae']:.4f}")
print(f"Adjacent Accuracy:          {test_results['eval_adjacent_accuracy']:.4f}")
print(f"Quadratic Weighted Kappa:   {test_results['eval_quadratic_weighted_kappa']:.4f}")

# Performance Metrics
print("\nPERFORMANCE METRICS:")
print(f"Inference Time:             {inference_time_sec:.4f} sec")
print(f"Model Size:                 {model_size_mb:.1f} MB")

# Per-Class Detailed Analysis (PRESERVED)
print("\nPER-CLASS PERFORMANCE ANALYSIS:")
sentiment_classes = ["extremely_negative", "negative", "neutral", "positive", "extremely_positive"]
class_display_names = ["Extremely Negative", "Negative", "Neutral", "Positive", "Extremely Positive"]

for i, (class_key, class_name) in enumerate(zip(sentiment_classes, class_display_names)):
    print(f"\n{class_name}:")
    print(f"  F1-Score:   {test_results.get(f'eval_f1_{class_key}', 0):.4f}")
    print(f"  Precision:  {test_results.get(f'eval_precision_{class_key}', 0):.4f}")
    print(f"  Recall:     {test_results.get(f'eval_accuracy_{class_key}', 0):.4f}")
    print(f"  Accuracy:   {test_results.get(f'eval_accuracy_{class_key}', 0):.4f}")

# Performance Analysis (PRESERVED + ENHANCED)
print("\nPERFORMANCE INSIGHTS:")
print(f"• MAE {test_results['eval_mae']:.2f}: On average off by {test_results['eval_mae']:.2f} sentiment levels")
print(f"• Adjacent Accuracy {test_results['eval_adjacent_accuracy']:.1%}: Predictions within 1 sentiment level")
print(f"• QWK {test_results['eval_quadratic_weighted_kappa']:.3f}: {'Excellent' if test_results['eval_quadratic_weighted_kappa'] > 0.8 else 'Good' if test_results['eval_quadratic_weighted_kappa'] > 0.6 else 'Moderate'} ordinal agreement")
# Added inference speed and efficiency insights
print(f"• Inference Speed: {1/inference_time_sec:.1f} predictions per second")
print(f"• Model Efficiency: {model_size_mb:.1f} MB storage required")

# Class Performance Analysis (PRESERVED)
f1_scores = [test_results.get(f'eval_f1_{class_key}', 0) for class_key in sentiment_classes]
best_class_idx = f1_scores.index(max(f1_scores))
worst_class_idx = f1_scores.index(min(f1_scores))

print(f"\nCLASS-SPECIFIC INSIGHTS:")
print(f"• Best performing class: {class_display_names[best_class_idx]} (F1: {f1_scores[best_class_idx]:.4f})")
print(f"• Most challenging class: {class_display_names[worst_class_idx]} (F1: {f1_scores[worst_class_idx]:.4f})")

# COVID-specific insights (PRESERVED)
extreme_avg = (f1_scores[0] + f1_scores[4]) / 2  # extremely negative + extremely positive
moderate_avg = (f1_scores[1] + f1_scores[3]) / 2  # negative + positive
neutral_score = f1_scores[2]

print(f"\nCOVID SENTIMENT INSIGHTS:")
print(f"• Extreme emotions (avg F1: {extreme_avg:.3f}): {'Challenging' if extreme_avg < 0.7 else 'Well-handled'}")
print(f"• Moderate emotions (avg F1: {moderate_avg:.3f}): {'Needs work' if moderate_avg < 0.75 else 'Good performance'}")
print(f"• Neutral sentiment (F1: {neutral_score:.3f}): {'Difficult to detect' if neutral_score < 0.8 else 'Well-identified'}")

# Final Summary with Key Metrics
print(f"\n📊 FINAL SUMMARY:")
print(f"F1: {test_results['eval_f1']:.4f} | QWK: {test_results['eval_quadratic_weighted_kappa']:.4f} | Inference: {inference_time_sec:.4f}s | Size: {model_size_mb:.1f}MB")

/tmp/ipython-input-3255646991.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  test_trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


FINAL TEST SET EVALUATION - COMPREHENSIVE ANALYSIS
{'eval_loss': 0.5976839065551758, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.833820093457944, 'eval_f1': 0.8329810902269634, 'eval_precision': 0.8310268173965134, 'eval_recall': 0.8477128360447125, 'eval_f1_extremely_negative': 0.8459770114942529, 'eval_precision_extremely_negative': 0.8, 'eval_recall_extremely_negative': 0.8975609756097561, 'eval_accuracy_extremely_negative': 0.8975609756097561, 'eval_f1_negative': 0.8036410923276983, 'eval_precision_negative': 0.8524137931034482, 'eval_recall_negative': 0.7601476014760148, 'eval_accuracy_negative': 0.7601476014760148, 'eval_f1_neutral': 0.8392101551480959, 'eval_precision_neutral': 0.7839262187088274, 'eval_recall_neutral': 0.9028831562974203, 'eval_accuracy_neutral': 0.9028831562974203, 'eval_f1_positive': 0.8244358833241607, 'eval_precision_positive': 0.8679026651216686, 'eval_recall_positive': 0.7851153039832285, 'eval_accuracy_positive': 0.7851153039832285, 'eval_f